In [1]:
import reservoirflow as rf
import numpy as np

In [2]:
rf.scalers.Dummy([10, 10])

{}

In [3]:
dir(rf.scalers)

['Dummy',
 'MinMax',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_dummy',
 '_minmax',
 '_scaler']

In [4]:
dir(rf.models)

['BlackOil',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_black_oil',
 '_model']

In [5]:
dir(rf)

['FACTORS',
 'UNITS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_base',
 'fluids',
 'grids',
 'models',
 'scalers',
 'utils',
 'wells']

In [6]:
rf.__version__

'0.0.1'

In [7]:
rf._base.UNITS

{'field': {'transmissibility': 'STB/D-psi',
  'error': 'STB/D',
  'pressure': 'Psia',
  'time': 'Days',
  'rate': 'STB/Day'},
 'metric': {'transmissibility': 'M3/D-bar', 'error': 'M3/D'}}

In [8]:
dir(rf)

['FACTORS',
 'UNITS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_base',
 'fluids',
 'grids',
 'models',
 'scalers',
 'utils',
 'wells']

In [9]:
rf.UNITS

{'field': {'transmissibility': 'STB/D-psi',
  'error': 'STB/D',
  'pressure': 'Psia',
  'time': 'Days',
  'rate': 'STB/Day'},
 'metric': {'transmissibility': 'M3/D-bar', 'error': 'M3/D'}}

In [10]:
rf.fluids.SinglePhase()

{'unit': 'field', 'units': {'transmissibility': 'STB/D-psi', 'error': 'STB/D', 'pressure': 'Psia', 'time': 'Days', 'rate': 'STB/Day'}, 'factors': {'transmissibility conversion': 0.001127, 'gravity conversion': 0.00021584000000000002, 'gravitational acceleration': 32.174, 'volume conversion': 5.614583}, 'dtype': 'double', 'verbose': False, 'rho': 0, 'g': 0.0, 'comp': 0, 'comp_type': 'incompressible'}

In [11]:
type(rf.scalers.MinMax((1, 1)))

reservoirflow.scalers._minmax.MinMax

In [12]:
def create_model_example_7_1():
    grid = rf.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = rf.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = rf.models.BlackOil(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model


def create_model():
    grid = rf.grids.Cartesian(
        nx=3,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=20,
        phi=0.27,
        kx=1,
        ky=1,
        kz=0.1,
        comp=1 * 10**-6,
        dtype="double",
    )
    fluid = rf.fluids.SinglePhase(
        mu=0.5, B=1, rho=50, comp=1 * 10**-5, dtype="double"
    )
    model = rf.models.BlackOil(
        grid, fluid, pi=6000, dt=5, start_date="10.10.2018", dtype="double"
    )

    n_samples = 1
    cells_id = model.grid.get_cells_id(False, True)[-1].flatten()
    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=-300, pwf=100, s=1.5, r=3.5)

    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=100, s=0, r=3.5)

    return model


model = create_model()
model.run(10, isolver="cgs")

[info] Simulation run started: 10 timesteps.


[step]: 10 - : 100%|██████████| 10/10 [00:00<00:00, 54.97steps/s]

[info] Simulation run of 10 steps finished in 0.2 seconds.
[info] Material Balance Error: 2.020605904817785e-14.


In [13]:
model.units

{'transmissibility': 'STB/D-psi',
 'error': 'STB/D',
 'pressure': 'Psia',
 'time': 'Days',
 'rate': 'STB/Day'}

In [14]:
arr = model.grid.get_cells_center(True, False)
print(arr.shape)
print(arr.min(axis=0))
arr

(5, 3)
[150. 175.  10.]


array([[ 150.,  175.,   10.],
       [ 450.,  175.,   10.],
       [ 750.,  175.,   10.],
       [1050.,  175.,   10.],
       [1350.,  175.,   10.]])

In [15]:
np.count_nonzero(np.array([1, 2, 0, 0]) == 0)

2

In [16]:
scaler = rf.scalers.MinMax((-1, 1)).fit(arr, axis=0)
scaler.transform(arr)

C:\Users\abugrzka\Projects\reservoirflow\reservoirflow\scalers\_minmax.py:38: RuntimeWarning: invalid value encountered in true_divide
  self.vmax - self.vmin


array([[-1. ,  nan,  nan],
       [-0.5,  nan,  nan],
       [ 0. ,  nan,  nan],
       [ 0.5,  nan,  nan],
       [ 1. ,  nan,  nan]])

In [17]:
t = np.arange(0, (model.tstep + 1) * model.dt, model.dt)
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [18]:
t

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

In [19]:
t, x = model.get_domain(False, True)
t, x

(array([[ 0],
        [ 5],
        [10],
        [15],
        [20],
        [25],
        [30],
        [35],
        [40],
        [45],
        [50]]),
 array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]))

In [20]:
rf.scalers.MinMax((-1, 1)).fit_transform(x, axis=0)

array([[-1. ],
       [-0.5],
       [ 0. ],
       [ 0.5],
       [ 1. ]])

In [21]:
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([[0. ],
       [0.1],
       [0.2],
       [0.3],
       [0.4],
       [0.5],
       [0.6],
       [0.7],
       [0.8],
       [0.9],
       [1. ]])

In [22]:
model.get_centers(False)

(array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]),
 ['x'])

In [23]:
config = {"time": ["MinMax", (0, 1)], "space": ["minMax"]}

In [24]:
model.set_scalers({"time": ["minmax", (-10, 10)], "rate": ["minmax", None]})
model.scalers_dict

{'time': ['MinMax', (-10, 10)],
 'space': ['MinMax', (-1, 1)],
 'pressure': ['MinMax', (-1, 1)],
 'rate': [None, None]}

In [25]:
model.update_scalers(False)
model.pressures_scaler.transform(model.pressures[1])
# model.space_scaler.transform(model.get_centers(False)[0])

array([0.41367714, 0.58787536, 0.37956372, 0.02714583, 0.41367714])

In [26]:
model.get_df(
    scale=True,
    melt=False,
    boundary=True,
    units=True,
    columns=["time", "cells", "wells"],
)

,Time [Scaled],Q1 [Scaled],Q3 [Scaled],P0 [Scaled],P1 [Scaled],P2 [Scaled],P3 [Scaled],P4 [Scaled],Qw3 [Scaled],Qw1 [Scaled],Pwf3 [Scaled],Pwf1 [Scaled]
Step,,,,,,,,,,,,
0,-10.0,0.0,0.000000,0.413677,0.413677,0.413677,0.413677,0.413677,0.000000,0.0,0.413677,0.413677
1,-8.0,100.0,-210.261660,0.413677,0.587875,0.379564,0.027146,0.413677,-210.261660,100.0,-2.516126,1.534475
2,-6.0,100.0,-187.135552,0.413677,0.719752,0.332658,-0.252582,0.413677,-187.135552,100.0,-2.516126,1.666351
3,-4.0,100.0,-170.190252,0.413677,0.816978,0.283514,-0.457548,0.413677,-170.190252,100.0,-2.516126,1.763578
4,-2.0,100.0,-157.628510,0.413677,0.886682,0.236949,-0.609491,0.413677,-157.628510,100.0,-2.516126,1.833281
5,0.0,100.0,-148.213227,0.413677,0.934958,0.194812,-0.723376,0.413677,-148.213227,100.0,-2.516126,1.881558
6,2.0,100.0,-141.073776,0.413677,0.966863,0.157460,-0.809733,0.413677,-141.073776,100.0,-2.516126,1.913463
7,4.0,100.0,-135.598989,0.413677,0.986372,0.124604,-0.875955,0.413677,-135.598989,100.0,-2.516126,1.932972
8,6.0,100.0,-131.349563,0.413677,0.996614,0.095694,-0.927355,0.413677,-131.349563,100.0,-2.516126,1.943214
